In [461]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# carga de datos 
all_data = pd.read_csv("Actualizado_DnD5e_Monsters_Stats.csv")
# seleccion de las columnas de entradas (todas menos la última)
all_inputs = all_data.iloc[:, :-1].values
# selecciono la columna de salida (la última)
all_outputs = all_data.iloc[:, -1].values

# División del conjunto de datos en entrenamiento y prueba (1/3 de prueba)
X_train, X_test, Y_train, Y_test = train_test_split(all_inputs, all_outputs, test_size=1/3)
n = X_train.shape[0] # número de registros de entrenamiento

# Pesos y sesgos iniciados aleatoriamente para el primer forward propagation 
np.random.seed(10) #Con una semilla para reproductibilidad
w_hidden = np.random.rand(4, 8) *2 - 1# hay 4 nodos en la  capa oculta y 8 entradas, cada una con su peso
w_output = np.random.rand(1, 4) *2 - 1# Hay 1 nodo de salida y 4 entradas de cada uno de los nodos de la capa anterior

b_hidden = np.random.rand(4, 1) *2 - 1# Al haber 4 nodos, cada uno tiene su propio sesgo
b_output = np.random.rand(1, 1) *2 - 1# El nodo de salida tiene su propio sesgo

# Funciones de activacion
relu = lambda x: np.maximum(x, 0) #Para la capa oculta
logistic = lambda x: 1 / (1 + np.exp(-x)) #Para la capa de salida


def forward_prop(X): # Forward propagation con una capa oculta y una capa de salida
    Z1 = w_hidden @ X + b_hidden
    A1 = relu(Z1)
    Z2 = w_output @ A1 + b_output
    A2 = logistic(Z2)
    return Z1, A1, Z2, A2


In [462]:
# Calculo de precisión de los datos de prueba (previo a backpropagation)

test_predictions = forward_prop(X_test.transpose())[3] 
# Obtengo A2 (Si se coloca 0, 1 o 2 se pueden obtener Z1,A1 o Z2 respectivamente)
test_comparisons = np.equal((test_predictions >= .5).flatten().astype(int), Y_test)#Comparación de predicciones con reales
accuracy = sum(test_comparisons.astype(int) / X_test.shape[0]) # Calculo de precisión
print("ACCURACY: ", accuracy)


ACCURACY:  0.49650349650349623


In [463]:
#Back propagation

# Tasa de aprendizaje
L = 0.01

# Derivadas de las funciones de activación
d_relu = lambda x: x > 0
d_logistic = lambda x: np.exp(-x) / (1 + np.exp(-x)) ** 2

# Cálculo de derivadas para obtener la derivada del costo con respecto a cada elemento
def backward_prop(Z1, A1, Z2, A2, X, Y):
    # Cálculo de cada parte de la derivada utilizando la regla de la cadena
    dC_dA2 = 2 * A2 - 2 * Y
    dA2_dZ2 = d_logistic(Z2)
    dZ2_dA1 = w_output
    dZ2_dW2 = A1
    dZ2_dB2 = 1
    dA1_dZ1 = d_relu(Z1)
    dZ1_dW1 = X
    dZ1_dB1 = 1

    # Derivadas del costo con respecto a W2, B2, A1, W1 y B1
    dC_dW2 = dC_dA2 @ dA2_dZ2 @ dZ2_dW2.T

    dC_dB2 = dC_dA2 @ dA2_dZ2 * dZ2_dB2

    dC_dA1 = dC_dA2 @ dA2_dZ2 @ dZ2_dA1

    dC_dW1 = dC_dA1 @ dA1_dZ1 @ dZ1_dW1.T

    dC_dB1 = dC_dA1 @ dA1_dZ1 * dZ1_dB1

    return dC_dW1, dC_dB1, dC_dW2, dC_dB2

In [464]:
# Descenso de gradiente estocástico
iteraciones = 100_000

for i in range(iteraciones):
    # seleccionar aleatoriamente uno de los datos de entrenamiento
    idx = np.random.choice(n, 1, replace=False) 
    X_sample = X_train[idx].transpose()
    Y_sample = Y_train[idx]

    # pasar datos seleccionados aleatoriamente a través de la red neuronal
    Z1, A1, Z2, A2 = forward_prop(X_sample)

    # distribuir error a través de la retropropagación
    # y devolver pendientes para pesos y sesgos
    dW1, dB1, dW2, dB2 = backward_prop(Z1, A1, Z2, A2, X_sample, Y_sample)

    # actualizar pesos y sesgos
    w_hidden -= L * dW1
    b_hidden -= L * dB1
    w_output -= L * dW2
    b_output -= L * dB2
    

In [465]:
# Calculo de precisión de entrenamiento
train_predictions = forward_prop(X_train.transpose())[3]
train_comparisons = np.equal((train_predictions >= .5).flatten().astype(int), Y_train)
train_accuracy = sum(train_comparisons.astype(int)) / X_train.shape[0]

print(f"Train Accuracy: {train_accuracy}")


# Calculo de precisión de prueba
test_predictions = forward_prop(X_test.transpose())[3]  
test_comparisons = np.equal((test_predictions >= .5).flatten().astype(int), Y_test)
test_accuracy = sum(test_comparisons.astype(int)) / X_test.shape[0]

print(f"Test Accuracy: {test_accuracy}")


Train Accuracy: 0.9403508771929825
Test Accuracy: 0.9300699300699301


In [466]:
# Pedir datos para hacer forward propagation con la red entrenada
# En formato diccionario para hacer un Dataframe
def pedir_datos():
    armor_class = int(input("Armor Class: "))
    hit_points = int(input("Hit Points: "))
    str = int(input("Strength: "))
    dex = int(input("Dexterity: "))
    con = int(input("Constitution: "))
    intelligence = int(input("Intelligence: "))
    wis = int(input("Wisdom: "))
    cha = int(input("Charisma: "))

    return {
        'Armor Class': armor_class,
        'Hit Points': hit_points,
        'STR': str,
        'DEX': dex,
        'CON': con,
        'INT': intelligence,
        'WIS': wis,
        'CHA': cha
    }
    

#Función de Forward propagation
def predict_probability(nuevos_valores):
    X= (nuevos_valores.values).T
    Z1, A1, Z2, A2 = forward_prop(X)
    return A2

# Si A2 es más cercano a 1, significa que el monstruo es un jefe
def predecir_si_jefe(nuevos_valores):
    output_values = predict_probability(nuevos_valores)
    if output_values > .5:
        return "Jefe"
    else:
        return "Monstruo común"


In [467]:
datos_usuario = pd.DataFrame([pedir_datos()])# Pedir datos al usuario y convertirlo a dataframe
minMax = pd.read_csv('minMax.csv').T #Al estar en archivos distintos, fue necesario pasar por un DF los min y max

#Normalizo los datos de usuario
for column in datos_usuario:
    min_val = minMax[0][column]
    max_val = minMax[1][column]
    datos_usuario[column] = (datos_usuario[column] - min_val) / (max_val - min_val)

predecir_si_jefe(datos_usuario)

'Jefe'